In [2]:
import pandas as pd
import re
import scrapy
from scrapy.http import HtmlResponse
import requests

In [21]:
# Obter os links e metados para as páginas de alunos
cos_students_url = "https://www.cos.ufrj.br/index.php/pt-BR/organizacao-aluno"

# Pegando HTML da página
links_req = requests.get(cos_students_url)
links_response = response = HtmlResponse(url='', body=links_req.text, encoding='utf-8')
# Selecionar tabela de alunos pelo xpath
student_links_xpath = '//*[@id="list_17_com_fabrik_17"]/tbody'
links_html_tbl = links_response.xpath(student_links_xpath).extract()

# Para obter links e metadados aplicamos regEx no texto da tabela (limpando as strings, quando necessário)
links_text = links_html_tbl[0]
links_text = links_text.replace("\n", "")
links_pattern = '<a href=\"(.+?)\">(.*?)</a>(.*?)</td>'
links_content = re.findall(links_pattern, links_text)

In [39]:
# iteramos nos dados obtidos para salvar os alunos e seus metadados
students_data = []
cos_url = "https://www.cos.ufrj.br"


for content in links_content:
    link_part = content[0]
    study_class = content[1].strip()
    students_number = content[2].strip()[1:3]

    # excluindo turmas anteriores por terem padrão diferente
    if not 'Anteriores' in study_class:
        class_url = f"{cos_url}{link_part}"
        class_req = requests.get(class_url)
        class_resp = HtmlResponse(url='', body=class_req.text, encoding='utf-8')
        students_table_xpath = '//*[@id="list_18_com_fabrik_18"]/tbody'
        students_table = class_resp.xpath(students_table_xpath).extract()
        students_text = students_table[0]
        students_text = students_text.replace("\n", "")
        students_text = students_text.replace("\t", "")
        studentds_pttrn = 'href=\"\/index\.php\/pt-BR\/pessoas\/details\/.*?>(.+?)</a>'
        candidates = re.findall(studentds_pttrn, students_text)
        students = [{'nome_aluno': s.strip(), 'turma': study_class, 'alunos_esperados': students_number} for s in candidates if '<' not in s]
        students_data.extend(students)

In [41]:
# transformando em dataframe para checar se o numero de alunos bate com o esperado
students_df = pd.DataFrame(students_data)
students_df.loc[:, 'alunos_esperados'] = students_df['alunos_esperados'].map(int)
check = students_df.groupby('turma').agg({'nome_aluno': 'count', 'alunos_esperados': 'mean'}).reset_index()
check.loc[:, 'status'] = check['nome_aluno'] == check['alunos_esperados']
check

,turma,nome_aluno,alunos_esperados,status
0,Alunos de Doutorado - Turma 2011,27,27,True
1,Alunos de Doutorado - Turma 2012,30,30,True
2,Alunos de Doutorado - Turma 2013,27,27,True
3,Alunos de Doutorado - Turma 2014,24,24,True
4,Alunos de Doutorado - Turma 2015,36,36,True
5,Alunos de Doutorado - Turma 2016,26,26,True
6,Alunos de Doutorado - Turma 2017,31,31,True
7,Alunos de Doutorado - Turma 2018,24,24,True
8,Alunos de Doutorado - Turma 2019,10,10,True
9,Alunos de Doutorado - Turma 2020,18,18,True


In [49]:
# salvando o nome dos alunos em txt
with open("todos_os_alunos.txt", 'w') as file:
        for student in students_df['nome_aluno'].values:
            file.write(student+'\n')
